# Master's thesis - Lukas Meuris - graphCast evaluation

This notebook contains the code to evaluate the models. 


In [1]:
#!pip install git+https://github.com/google-research/weatherbench2.git

  Cloning https://github.com/google-research/weatherbench2.git to c:\users\lukas\appdata\local\temp\pip-req-build-nx0j_vv6
  Resolved https://github.com/google-research/weatherbench2.git to commit c6c4a1ad6f596714051be544bd0e955f8cc4ff6d
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/89.7 kB ? eta -:--:--
     ---------------------- ----------------- 51.2/89.7 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 89.7/89.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
     ---------------------

  Running command git clone --filter=blob:none --quiet https://github.com/google-research/weatherbench2.git 'C:\Users\lukas\AppData\Local\Temp\pip-req-build-nx0j_vv6'
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\lukas\\anaconda3\\envs\\mtlmenv\\lib\\site-packages\\pandas\\_libs\\algos.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
#import apache_beam   # Needs to be imported separately to avoid TypingError
import weatherbench2

# Specify input datasets
!! need to download the data from the bucket using google colab, before it can be used.


In [28]:
# empty for now.

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/weatherbench2?projection=noAcl&prettyPrint=false: lukas-494@resonant-sunset-420111.iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket. Permission 'storage.buckets.get' denied on resource (or it may not exist).

### Set up WeatherBench configuration

Next, we will define a bunch of configuration instances to specify exactly what we want to evaluate.

In [3]:
from weatherbench2 import config

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

# data configuration
The file paths are defined in a Paths config object, alongside an output directory:

In [ ]:
paths = config.Paths(
    forecast=forecast_path,
    obs=obs_path,
    output_dir='./',   # Directory to save evaluation results
)

In addition, we specify a Selection object that selects the variables and time period to be evaluated.

In [ ]:
selection = config.Selection(
    variables=[
        'geopotential',
        '2m_temperature'
    ],
    levels=[500, 700, 850],
    time_slice=slice('2020-01-01', '2020-12-31'),
)

Together they make up the Data config:

In [ ]:
data_config = config.Data(selection=selection, paths=paths)

#### Evaluation configuration

Next, we can defined which evaluation we want to run. To do so, we can define a dictionary of `config.Eval`s, each of which will be evaluated separately and saved to a different file. Eval instances contain the metrics objects, defined in metrics.py.

Note that for ACC, we additionally need to pass the climatology opened earlier.

from weatherbench2.metrics import MSE, ACC

eval_configs = {
  'deterministic': config.Eval(
      metrics={
          'mse': MSE(), 
          'acc': ACC(climatology=climatology) 
      },
  )
}

The evaluation configs also have an option to evaluate particular regions, such as a geographical lat-lon box. These are defined as region objects defined in regions.py. All regions will be evaluated separately and saved as an additional dimension in the dataset. If no region is specified, evaluation will be done globally.

In [ ]:
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion

regions = {
    'global': SliceRegion(),
    'tropics': SliceRegion(lat_slice=slice(-20, 20)),
    'extra-tropics': ExtraTropicalRegion(),
}

eval_configs = {
  'deterministic': config.Eval(
      metrics={
          'mse': MSE(), 
          'acc': ACC(climatology=climatology) 
      },
      regions=regions
  )
}

### Evaluate

Now, we are already done and can run the evaluation. We can do so in memory for smaller datasets or as a Beam pipeline (recommended for anything larger than 64x32 resolution).

In [ ]:
from weatherbench2.evaluation import evaluate_in_memory

In [ ]:
evaluate_in_memory(data_config, eval_configs)   # Takes around 5 minutes

### Results
Results are saved as NetCDF files under the name of the eval config.

In [ ]:
results = xr.open_dataset('./deterministic.nc')
results

Note that to compute the RMSE, we follow ECMWF's convention by taking the square root after the time mean. To do this in WB2, first compute the MSE and then take the square root of the saved MSE results

In [ ]:
results = xr.concat(
    [
    results,
    results.sel(metric=['mse']).assign_coords(metric=['rmse']) ** 0.5
    ],
    dim='metric'
)

In [ ]:
results['geopotential'].sel(metric='rmse', level=500, region='global').plot();